In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf

2022-12-16 00:49:03.052190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 00:49:03.101627: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-12-16 00:49:04.501703: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:49:04.505525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:49:04.505668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
from models import control_ma,control_tapper,complex,complex_long,our_simp_comp,our_simple,complex_wide

In [4]:
prefix = "data/final_data"

In [5]:
training_run=1

#data_list = [{"suffix":"unsup_5","water_suppression":False,"snr":5}]
#data_list = [{"suffix":"unsup_2_5","water_suppression":False,"snr":2.5}]
#data_list = [{"suffix":"unsup_10","water_suppression":False,"snr":10}]
data_list = [{"suffix":"sup_5","water_suppression":True,"snr":5}]
#data_list = [{"suffix":"sup_2_5","water_suppression":True,"snr":2.5}]
#data_list = [{"suffix":"sup_10","water_suppression":True,"snr":10}]

#data_list = [{"suffix":"unsup_x","water_suppression":False,"snr":0}]
#data_list = [{"suffix":"sup_x","water_suppression":False,"snr":0}]


model_list=[
    {
        "model_name":"control_ma",
        "pipe_class":control_ma.Control,
        "single_net":False
    },
    {
        "model_name":"control_tapper",
        "pipe_class":control_tapper.Control,
        "single_net":False
    },
    {
        "model_name":"complex",
        "pipe_class":complex.ComplexModel,
        "single_net":False
    }
]

model_list=[
    {
        "model_name":"our_simp",
        "pipe_class":our_simple.ComplexModel,
        "single_net":False
    },
    {
        "model_name":"our_simp_comp",
        "pipe_class":our_simp_comp.ComplexModel,
        "single_net":False
    },
    {
        "model_name":"complex",
        "pipe_class":complex.ComplexModel,
        "single_net":False
    }
]


model_list=[
    {
        "model_name":"control_ma",
        "pipe_class":control_ma.Control,
        "single_net":False
    },
    {
        "model_name":"control_tapper",
        "pipe_class":control_tapper.Control,
        "single_net":False
    },
    {
        "model_name":"complex",
        "pipe_class":complex.ComplexModel,
        "single_net":False
    },
    {
        "model_name":"complex_wide",
        "pipe_class":complex_wide.ComplexModel,
        "single_net":False
    },
    {
        "model_name":"our_simp",
        "pipe_class":our_simple.ComplexModel,
        "single_net":False
    },
    {
        "model_name":"our_simp_comp",
        "pipe_class":our_simp_comp.ComplexModel,
        "single_net":False
    }
    ]


In [6]:
dfs = []

prefix = "data/final_data"
saved_models_prefix="models/saved_weights"
post_suffix = f"_{training_run}"

In [7]:
for data_info in data_list:

    suffix = data_info["suffix"]
    water_suppression = data_info["water_suppression"]
    snr = data_info["snr"]

    
    x = np.load(f"{prefix}/x_data_1024_{suffix}.npy")[:,:,:,:]
    x_phase = np.load(f"{prefix}/x_data_phase_1024_{suffix}.npy")[:,:,:,:]
    y = np.load(f"{prefix}/y_data_1024_{suffix}.npy")[:,:,:,:]


    indexes = np.arange(x.shape[0], dtype = int)
    np.random.shuffle(indexes)

    x = x[indexes]
    x_phase = x_phase[indexes]
    y = y[indexes]



    for model_info in model_list:

        print(model_info["model_name"])
        
        pipe = model_info["pipe_class"]()
        if model_info["single_net"]:
            pipe.load_data(x.copy(),y.copy())
        else:
            pipe.load_data(x.copy(),x_phase.copy(),y.copy())
        pipe.train(epoch_count=100,scheduler_epoch=25)
        df = pipe.get_test_results()
        df["model"]=model_info["model_name"]
        df["water_suppression"]=water_suppression
        df["snr"]=snr
        df["training_run"]=training_run
        dfs.append(df)
        if model_info["single_net"]:
            pipe.save_model_weights(f"{saved_models_prefix}/{model_info['model_name']}_{suffix}{post_suffix}.h5")
        else:
            pipe.save_model_weights(f"{saved_models_prefix}/{model_info['model_name']}_freq_{suffix}{post_suffix}.h5",f"{saved_models_prefix}/{model_info['model_name']}_phase_{suffix}{post_suffix}.h5")
        del pipe
    
    df = pd.concat(dfs,sort=False)
    df.to_csv(f"study_results/{suffix}{post_suffix}.csv")
    



control_ma


2022-12-16 00:49:24.807600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 00:49:24.808720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:49:24.808898: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 00:49:24.808971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one

Epoch 1/100


2022-12-16 00:49:26.122983: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


  1/800 [..............................] - ETA: 22:06 - loss: 75.8865

2022-12-16 00:49:26.930609: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


800/800 [==============================] - 4s 3ms/step - loss: 1.4283 - val_loss: 0.2981 - lr: 0.0010
Epoch 2/100
800/800 [==============================] - 2s 3ms/step - loss: 0.1657 - val_loss: 0.1813 - lr: 0.0010
Epoch 3/100
800/800 [==============================] - 2s 3ms/step - loss: 0.1090 - val_loss: 0.1483 - lr: 0.0010
Epoch 4/100
800/800 [==============================] - 2s 3ms/step - loss: 0.1004 - val_loss: 0.1332 - lr: 0.0010
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0860 - val_loss: 0.1392 - lr: 0.0010
Epoch 6/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0801 - val_loss: 0.0929 - lr: 0.0010
Epoch 7/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0730 - val_loss: 0.0880 - lr: 0.0010
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.0693 - val_loss: 0.0960 - lr: 0.0010
Epoch 9/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0604 - val_loss: 0.0835